<a href="https://colab.research.google.com/github/katekuz/univ/blob/master/continents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_excel('/content/drive/MyDrive/covid_data/owid-covid-data.xlsx')
worksheet  = gc.open('ukr').get_worksheet(1)
#dfUkr = pd.read_excel('/content/drive/MyDrive/covid_data/ukr.gsheet')
rows = worksheet.get_all_values()
print(rows)

dfUkr = pd.DataFrame.from_records(rows)
#pd.set_option('display.max_rows', None)
dfUkr.drop([1,3,4,5,6,7,8] , axis=1, inplace = True)
dfUkr.drop([0], inplace = True)

dfUkr.head()

[['Дата', 'Кількість підтверджених випадків', 'Кількість нових підтверджених випадків за добу', 'Одужало всього', 'Померло всвього', 'Одужало за добу', 'Померло за добу', 'Медиків всього', 'Медиків за добу'], ['29.02.2020', '1', '1', '', '', '', '', '', ''], ['06.03.2020', '3', '2', '', '', '', '', '', ''], ['09.03.2020', '3', '0', '', '', '', '', '', ''], ['10.03.2020', '4', '1', '', '', '', '', '', ''], ['12.03.2020', '6', '2', '', '', '', '', '', ''], ['13.03.2020', '10', '4', '', '', '', '', '', ''], ['14.03.2020', '13', '3', '', '', '', '', '', ''], ['15.03.2020', '17', '4', '', '', '', '', '', ''], ['16.03.2020', '20', '3', '', '', '', '', '', ''], ['17.03.2020', '22', '2', '', '', '', '', '', ''], ['18.03.2020', '33', '11', '', '', '', '', '', ''], ['19.03.2020', '49', '16', '', '', '', '', '', ''], ['20.03.2020', '70', '21', '', '', '', '', '', ''], ['21.03.2020', '86', '16', '', '', '', '', '', ''], ['22.03.2020', '99', '13', '', '', '', '', '', ''], ['23.03.2020', '106', '7',

,0,2
1,29.02.2020,1
2,06.03.2020,2
3,09.03.2020,0
4,10.03.2020,1
5,12.03.2020,2


In [ ]:
dfUkr.set_axis(['date', 'Ukraine'], axis=1, inplace=True)
dfUkr

,date,Ukraine
1,29.02.2020,1
2,06.03.2020,2
3,09.03.2020,0
4,10.03.2020,1
5,12.03.2020,2
...,...,...
425,06.05.2021,8404
426,07.05.2021,8710
427,08.05.2021,5372
428,09.05.2021,2817


In [ ]:
dfUkr['Ukraine'] = pd.to_numeric(dfUkr['Ukraine'])
dfUkr['Ukraine'] = dfUkr['Ukraine']*100000/38085266

In [ ]:
#dfOld[dfOld['location'] == 'Andorra']

In [ ]:
#df[df['new_cases'] < 0]
#df[df['new_cases'].isna]
#df[df['location']=='World']['continent'].isna().all()
#pd.set_option('display.max_rows', None)
df[df['location'] == 'Sweden']

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
75661,SWE,Europe,Sweden,2020-02-01,1.0,1.0,NaN,NaN,NaN,NaN,0.099,0.099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,10099270.0,24.718,41.0,19.985,13.433,46949.283,0.5,133.982,4.79,18.8,18.9,NaN,2.22,82.8,0.945
75662,SWE,Europe,Sweden,2020-02-02,1.0,0.0,NaN,NaN,NaN,NaN,0.099,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,10099270.0,24.718,41.0,19.985,13.433,46949.283,0.5,133.982,4.79,18.8,18.9,NaN,2.22,82.8,0.945
75663,SWE,Europe,Sweden,2020-02-03,1.0,0.0,NaN,NaN,NaN,NaN,0.099,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,10099270.0,24.718,41.0,19.985,13.433,46949.283,0.5,133.982,4.79,18.8,18.9,NaN,2.22,82.8,0.945
75664,SWE,Europe,Sweden,2020-02-04,1.0,0.0,NaN,NaN,NaN,NaN,0.099,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,10099270.0,24.718,41.0,19.985,13.433,46949.283,0.5,133.982,4.79,18.8,18.9,NaN,2.22,82.8,0.945
75665,SWE,Europe,Sweden,2020-02-05,1.0,0.0,NaN,NaN,NaN,NaN,0.099,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,10099270.0,24.718,41.0,19.985,13.433,46949.283,0.5,133.982,4.79,18.8,18.9,NaN,2.22,82.8,0.945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76121,SWE,Europe,Sweden,2021-05-06,1002121.0,6526.0,4920.429,14158.0,7.0,22.286,99227.073,646.185,487.206,1401.884,0.693,2.207,1.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3563609.0,2769411.0,794198.0,77884.0,47106.0,35.29,27.42,7.86,4664.0,65.74,10099270.0,24.718,41.0,19.985,13.433,46949.283,0.5,133.982,4.79,18.8,18.9,NaN,2.22,82.8,0.945
76122,SWE,Europe,Sweden,2021-05-07,1007792.0,5671.0,4884.000,14173.0,15.0,17.857,99788.599,561.526,483.599,1403.369,1.485,1.768,1.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3679451.0,2852689.0,826762.0,115842.0,58937.0,36.43,28.25,8.19,5836.0,65.74,10099270.0,24.718,41.0,19.985,13.433,46949.283,0.5,133.982,4.79,18.8,18.9,NaN,2.22,82.8,0.945
76123,SWE,Europe,Sweden,2021-05-08,1007792.0,0.0,4884.000,14173.0,0.0,17.857,99788.599,0.000,483.599,1403.369,0.000,1.768,1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10099270.0,24.718,41.0,19.985,13.433,46949.283,0.5,133.982,4.79,18.8,18.9,NaN,2.22,82.8,0.945
76124,SWE,Europe,Sweden,2021-05-09,1007792.0,0.0,4884.000

In [ ]:
#dfCont['Asia'] = sum(df[df['continent']=='Asia'])
df.dropna(subset = ['iso_code','continent'], inplace=True)
df.fillna(0, inplace=True)

df['date1'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df.sort_values(by = ['location', 'date1'])    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!CHANGES
df.drop(columns=['date1'], inplace = True)
df = df.reset_index()
#df

In [ ]:
#DROP WITHOUT ISO
dates = df['date'].unique().tolist()
countries = df['location'].unique().tolist()
continents = df['continent'].unique().tolist()
countries_map = {}

In [ ]:
def summarize(a):
  x = 0 
  for i in a:
    x = x + i
  return x 

def correctOWID(df):  
  for index, row in df.iterrows():
      z = row['location']
      cases_country = row['new_cases']
      indexes_to_update = []
      #population_country = datecontcountdf['population'].sum()
      if(True and (cases_country == 0 or np.isnan(cases_country))):
          temp_index = index
          indexes_to_update = [index]
          while(df.at[temp_index, 'new_cases'] == 0 and df.at[temp_index, 'location'] == z):
            temp_index += 1 
            indexes_to_update = indexes_to_update + [temp_index]
          if(df.at[temp_index, 'location'] == z):
            summ = df.at[temp_index, 'new_cases']
            if(summ < 0):
              if(z in countries_map):
                last_days = countries_map[z]
                if(len(last_days[-7:]) == 0):
                    avg = 0
                else:
                    avg = summarize(last_days[-7:])/len(last_days[-7:])
                summ = round(avg)
            cases_country = round(summ / len(indexes_to_update))
            for i in indexes_to_update:
              df.at[i, 'new_cases'] = cases_country
          else:
            if(z in countries_map):
              last_days = countries_map[z]
              if(len(last_days[-7:]) == 0):
                  avg = 0
              else:
                  avg = summarize(last_days[-7:])/len(last_days[-7:])
              cases_country = round(avg)
            else:
              cases_country = 0
            #print(cases_country)
            df.at[index, 'new_cases'] = cases_country     
      else:
        if(cases_country < 0):        
          if(z in countries_map):
              last_days = countries_map[z]
              if(len(last_days[-7:]) == 0):
                  avg = 0
              else:
                  avg = summarize(last_days[-7:])/len(last_days[-7:])
              cases_country = round(avg)
          else:
              cases_country = 0
          #print(cases_country)
          df.at[index, 'new_cases'] = cases_country    
      if(z in countries_map):
          countries_map[z] = countries_map[z]+[cases_country]
      else:
          cases_country = 0    
          countries_map[z] = [cases_country] 
   
  return df

In [ ]:
df = correctOWID(df)
#df[df['new_cases']<0]

In [ ]:
df.to_excel("correctedOWID.xlsx", index = False)

In [ ]:
#====================================================================================================

In [ ]:
#SUM BY CONTINENT
data = []
df['date1'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df.sort_values(by = ['date1'])    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!CHANGES
dates = df['date'].unique().tolist()
countries = df['location'].unique().tolist()
continents = df['continent'].unique().tolist()
df.drop(columns=['date1'], inplace = True)
for x in dates:
    to_append = [x]
    datedf = df[df['date']==x]
    cases = []
    population = []
    #print(x)
    for y in continents:
        datecontdf = datedf[datedf['continent'] == y]
        countries = datecontdf.location.unique()
        cases = cases + [datecontdf['new_cases'].sum()]
        population = population + [datecontdf['population'].sum()]
        #print(y)
        to_append = to_append+[100000*cases[-1]/population[-1]]
    to_append = to_append+[datedf['new_cases'].sum()*100000/datedf['population'].sum()]
    #to_append = to_append+[datedf[datedf['location']== 'Ukraine']['new_cases'].sum()*100000/datedf[datedf['location']== 'Ukraine']['population'].sum()]
    data = data + [(to_append)]
data = pd.DataFrame(data, columns = ['date']+continents+['World'])#, 'Ukraine'
#data.to_excel("conts.xlsx")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
data

,date,North America,South America,Asia,Europe,Oceania,Africa,World
0,2020-01-01,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000
1,2020-01-02,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000
2,2020-01-03,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000
3,2020-01-04,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000
4,2020-01-05,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...
491,2021-05-06,11.101477,31.805920,11.375841,16.604761,0.457476,0.774036,11.106900
492,2021-05-07,11.081241,33.866053,10.974802,15.750409,0.327456,0.752226,10.894285
493,2021-05-08,8.576830,29.012148,10.837369,14.729126,0.327456,0.722498,10.249499
494,2021-05-09,5.763692,19.813101,9.882884,10.121295,0.305787,0.676189,8.504653


In [ ]:
data['date1'] = pd.to_datetime(data['date'])
#data['date1'] = data['date1'].dt.strftime('%d.%m.%Y')
dfUkr['date1'] = pd.to_datetime(dfUkr['date'], format='%d.%m.%Y')
#dfUkr['date1'] = dfUkr['date1'].dt.strftime('%d.%m.%Y')
dataMerged = pd.merge(data, dfUkr, how='outer', on='date1')

dates = df['date'].unique().tolist()
dataMerged.drop(columns=['date_x', 'date_y'], inplace = True)


In [ ]:
dataMerged.tail(10)

,North America,South America,Asia,Europe,Oceania,Africa,World,date1,Ukraine
486,10.299444,28.827461,10.846657,16.687312,0.187806,0.674471,10.555978,2021-05-01,13.375251
487,7.084449,18.026862,10.282186,10.882410,0.269670,0.531884,8.791426,2021-05-02,7.241646
488,11.426786,15.480967,9.971968,8.545835,0.170952,0.477806,8.558930,2021-05-03,6.490699
489,9.619308,31.602880,10.610526,14.611747,0.089087,0.601945,10.304124,2021-05-04,6.763771
490,10.688422,29.687941,11.214123,16.672005,0.488777,0.735794,10.862177,2021-05-05,15.853900
491,11.101477,31.805920,11.375841,16.604761,0.457476,0.774036,11.106900,2021-05-06,22.066276
492,11.081241,33.866053,10.974802,15.750409,0.327456,0.752226,10.894285,2021-05-07,22.869737
493,8.576830,29.012148,10.837369,14.729126,0.327456,0.722498,10.249499,2021-05-08,14.105192
494,5.763692,19.813101,9.882884,10.121295,0.305787,0.676189,8.504653,2021-05-09,7.396561
495,8.658293,17.611723,9.079549,8.114697,0.305787,0.537411,7.906493,2021-05-10,5.797518


In [ ]:
data[data['date']=='2021-01-09']

,date,North America,South America,Asia,Europe,Oceania,Africa,World,date1
374,2021-01-09,50.230443,23.614637,1.885153,34.198232,0.072434,2.681975,10.018445,2021-01-09


In [ ]:
dataMerged.set_index(['date1'],inplace=True)
dataMerged = dataMerged.rolling(window = 7).mean()

In [ ]:
dataMerged

,North America,South America,Asia,Europe,Oceania,Africa,World,Ukraine
date1,,,,,,,,
2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2021-05-06,10.493298,26.599566,10.795340,14.519048,0.264511,0.651786,10.203090,13.462649
2021-05-07,10.185875,27.042583,10.753729,14.250640,0.284461,0.649737,10.153403,13.523040
2021-05-08,9.939787,27.068967,10.752402,13.970899,0.304411,0.656598,10.109620,13.627317


In [ ]:
#for x in dfOut.columns:
#    dfOut[x] = dfOut[x].rolling(window = 7).mean()

In [ ]:
dataMerged.to_excel("conts.xlsx")

In [ ]:
#plt.figure(figsize=(25, 20))
#----------
plt.plot(dataMerged)#['date1'], dataMerged['Asia'], dataMerged['Ukraine'])
plt.legend(dataMerged.columns)
plt.xticks(np.arange(0, len(dataMerged)+1, 14))
plt.xticks(rotation = 90)

plt.show()
plt.savefig('continents.png')

ValueError: ignored

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 0 Axes>

In [ ]:
bigCountries = ['Ukraine', 'Sweden', 'France','Czech Republic','Spain','Portugal','Netherlands','Italy',
                'United Kingdom','Belgium','Poland','Romania','Greece','Russia','Germany']

In [ ]:
df['date1'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df.sort_values(by = ['date1'])    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!CHANGES
dates = df['date'].unique().tolist()
df.drop(columns=['date1'], inplace = True)
dfBigCountries = df[df['location'].isin(bigCountries)]
last7days = dates[-7:]
last14days = dates[-14:]
df7 = dfBigCountries[dfBigCountries['date'].isin(last7days)]
df7


,index,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
7611,8539,BEL,Europe,Belgium,2021-05-04,999627.0,2731.0,2941.857,24367.0,45.0,37.571,86251.952,235.642,253.836,2102.486,3.883,3.242,0.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65560.0,12762517.0,1101.203,5.657,48483.0,4.183,0.069,14.5,tests performed,4111271.0,3237048.0,883284.0,80895.0,69953.0,35.47,27.93,7.62,6036.0,54.63,11589616.0,375.564,41.8,18.571,12.849,42658.576,0.2,114.898,4.29,25.1,31.4,0.0,5.640,81.63,0.931
37284,39619,ITA,Europe,Italy,2021-05-04,4059821.0,9113.0,11187.000,121738.0,305.0,260.857,67146.845,150.723,185.026,2013.469,5.045,4.314,0.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,315506.0,59552161.0,984.955,5.218,282513.0,4.673,0.040,25.3,tests performed,21696095.0,15253627.0,6555723.0,444083.0,443514.0,35.88,25.23,10.84,7335.0,75.00,60461828.0,205.859,47.9,23.021,16.240,35220.084,2.0,113.151,4.78,19.8,27.8,0.0,3.180,83.51,0.892
70610,74336,ESP,Europe,Spain,2021-05-04,3544945.0,4515.0,6973.000,78399.0,106.0,77.714,75819.943,96.568,149.140,1676.812,2.267,1.662,0.91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0,17916566.0,12686341.0,5394315.0,485756.0,362672.0,38.32,27.13,11.54,7757.0,67.59,46754783.0,93.105,45.5,19.436,13.799,34272.360,1.0,99.403,7.17,27.4,31.4,0.0,2.970,83.56,0.904
60123,63405,POL,Europe,Poland,2021-05-04,2808052.0,2296.0,5716.857,68133.0,28.0,319.429,74195.611,60.666,151.053,1800.241,0.740,8.440,0.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30793.0,14145372.0,373.755,0.814,51842.0,1.370,0.110,9.1,people tested,0.0,0.0,0.0,0.0,223591.0,0.00,0.00,0.00,5908.0,71.30,37846605.0,124.027,41.8,16.763,10.202,27216.445,0.0,227.331,5.91,23.3,33.1,0.0,6.620,78.73,0.880
61906,65188,RUS,Europe,Russia,2021-05-04,4784497.0,7653.0,8463.571,109670.0,329.0,357.571,32785.245,52.441,57.996,751.502,2.254,2.450,0.97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,174309.0,130413043.0,893.641,1.194,268900.0,1.843,0.031,31.8,tests performed,20168443.0,12478994.0,7689449.0,57889.0,230197.0,13.82,8.55,5.27,1577.0,36.57,145934460.0,8.823,39.6,14.178,9.393,24765.954,0.1,431.297,6.18,23.4,58.3,0.0,8.050,72.58,0.824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61446,64728,ROU,Europe,Romania,2021-05-10,1066731.0,620.0,1199.143,29034.0,68.0,93.429,55450.080,32.228,62.333,1509.225,3.535,4.857,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,19237682.0,85.129,43.0,17.850,11.690,23313.199,5.7,370.946,9.74,22.9,37.1,0.0,6.892,76.05,0.828
61912,65194,RUS,Europe,Russia,2021-05-10,4832959.0,8338.0,8016.429,111740.0,315.0,342.714,33117.325,57.135,54.932,765.686,2.159,2.348,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193338.0,131924338.

In [ ]:
#-------------------------------------------
for index, row in df.iterrows():
    z = row['location']
    cases_country = row['new_cases']
    #population_country = datecontcountdf['population'].sum()
    if(cases_country < 0 or np.isnan(cases_country)):
        #print(cases_country)
        if(z in countries_map):
            last_days = countries_map[z]
            if(len(last_days[-7:]) == 0):
                avg = 0
            else:
                avg = sum(last_days[-7:])/len(last_days[-7:])
            cases_country = round(avg)
        else:
            avg = 0
        #print(cases_country)
        df.at[index, 'new_cases'] = cases_country    
    if(z in countries_map):
        countries_map[z] = countries_map[z]+[cases_country]
    else:
        cases_country = 0    
        countries_map[z] = [cases_country]   
